In [1]:
import glob
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import rocks
rocks.set_log_level("error")
import time as t


from astropy.time import Time
import requests
import io

import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import angular_separation

from fink_utils.sso import spins as finkus

# Get ZTF fit

In [2]:
# Local Configuration
data_fink = '/data/fink/tutorials/sso/spins/'
data_fink = './'

# To choose the definition of sets
# gold_silver_choice = 'julien'
gold_silver_choice = 'benoit'

In [3]:

# ZTF filters 1: g, 2: r
filters = {'1': 'g', '2': 'r'}
filters = {'1': 'g', '2': 'r', 3:'o', 4:'c'}

In [4]:
# data_hg1g2hyb = pd.read_parquet(f'{data_fink}data/sso_bft_50_inf_hybrid_bounds.parquet') # All
data_hg1g2hyb = pd.read_parquet(f'{data_fink}data/sso_bft_50_inf_hybrid_ATLAS.parquet') # Test ATLAS
data_hg1g2sp = pd.read_parquet(f'{data_fink}data/sso_bft_50_inf_hg1g2spins.parquet')
data_hg1g2 = pd.read_parquet(f'{data_fink}data/sso_bft_50_inf_hg1g2.parquet')
data_hg12 = pd.read_parquet(f'{data_fink}data/sso_bft_50_inf_hg12.parquet')
data_hg = pd.read_parquet(f'{data_fink}data/sso_bft_50_inf_hg.parquet')

In [5]:
len(data_hg), len(data_hg12),  len(data_hg1g2),  len(data_hg1g2sp),  len(data_hg1g2hyb)

(104242, 104242, 104242, 104242, 104242)

# Reformat solutions

In [6]:
# HG to data
names_numbers = rocks.identify(data_hg.ssnamenr)
data_hg['number'] = [nn[1] for nn in names_numbers]
data_hg['name'] = [nn[0] for nn in names_numbers]

data_hg['minphase'] = np.rad2deg(data_hg['params'].apply(lambda x: x['minphase']).values)
data_hg['maxphase'] = np.rad2deg(data_hg['params'].apply(lambda x: x['maxphase']).values)
data_hg['n_days'] = data_hg['ndays']
data_hg['n_obs'] = data_hg['nmeasurements']

data_hg['HG_H'] = data_hg['params'].apply(lambda x: x['H']).values
data_hg['HG_dH'] = data_hg['params'].apply(lambda x: x['errH']).values
data_hg['HG_G'] = data_hg['params'].apply(lambda x: x['G']).values
data_hg['HG_dG'] = data_hg['params'].apply(lambda x: x['errG']).values

data_hg['HG_rms'] = data_hg['params'].apply(lambda x: x['rms']).values
data_hg['HG_chi2red'] = data_hg['params'].apply(lambda x: x['chi2red']).values

data_hg = data_hg.drop(columns=[ 'nmeasurements', 'ndays', 'params'])

cond = data_hg.name.notna()
data_hg = data_hg[cond].reset_index(drop=True)

ERROR    [rocks] Could not identify '2002 XZ04'.

ERROR    [rocks] Could not identify '2012 LV07'.

ERROR    [rocks] Could not identify '2017 LO01'.

ERROR    [rocks] Could not identify '2001 BO02'.

ERROR    [rocks] Could not identify '2020 AV02'.

...

ERROR    [rocks] Could not identify '2003 VA03'.

ERROR    [rocks] Could not identify '2016 QP01'.

ERROR    [rocks] Could not identify '2007 TG08'.

ERROR    [rocks] Could not identify '2016 OQ02'.

ERROR    [rocks] Could not identify '2009 CC03'.

In [7]:
# HG1G2 to data
data_hg1g2['HG1G2_H'] = data_hg1g2['params'].apply(lambda x: x['H']).values
data_hg1g2['HG1G2_dH'] = data_hg1g2['params'].apply(lambda x: x['errH']).values
data_hg1g2['HG1G2_G1'] = data_hg1g2['params'].apply(lambda x: x['G1']).values
data_hg1g2['HG1G2_dG1'] = data_hg1g2['params'].apply(lambda x: x['errG1']).values
data_hg1g2['HG1G2_G2'] = data_hg1g2['params'].apply(lambda x: x['G2']).values
data_hg1g2['HG1G2_dG2'] = data_hg1g2['params'].apply(lambda x: x['errG2']).values

# data_hg1g2['HG1G2_minphase'] = np.rad2deg(data_hg1g2['params'].apply(lambda x: x['minphase']).values)
# data_hg1g2['HG1G2_maxphase'] = np.rad2deg(data_hg1g2['params'].apply(lambda x: x['maxphase']).values)
# data_hg1g2['HG1G2_n_days'] = data_hg1g2['ndays']
# data_hg1g2['HG1G2_n_obs'] = data_hg1g2['nmeasurements']
data_hg1g2['HG1G2_rms'] = data_hg1g2['params'].apply(lambda x: x['rms']).values
data_hg1g2['HG1G2_chi2red'] = data_hg1g2['params'].apply(lambda x: x['chi2red']).values

data_hg1g2 = data_hg1g2.drop(columns=['nmeasurements', 'ndays', 'params'])

In [8]:
# HG1G2spin to data
data_hg1g2sp['HG1G2sp_H'] = data_hg1g2sp['params'].apply(lambda x: x['H']).values
data_hg1g2sp['HG1G2sp_dH'] = data_hg1g2sp['params'].apply(lambda x: x['errH']).values
data_hg1g2sp['HG1G2sp_G1'] = data_hg1g2sp['params'].apply(lambda x: x['G1']).values
data_hg1g2sp['HG1G2sp_dG1'] = data_hg1g2sp['params'].apply(lambda x: x['errG1']).values
data_hg1g2sp['HG1G2sp_G2'] = data_hg1g2sp['params'].apply(lambda x: x['G2']).values
data_hg1g2sp['HG1G2sp_dG2'] = data_hg1g2sp['params'].apply(lambda x: x['errG2']).values

data_hg1g2sp['HG1G2sp_RA0'] = np.degrees(data_hg1g2sp['params'].apply(lambda x: x['RA0']).values)
data_hg1g2sp['HG1G2sp_dRA0'] = np.degrees(data_hg1g2sp['params'].apply(lambda x: x['errRA0']).values)
data_hg1g2sp['HG1G2sp_DEC0'] = np.degrees(data_hg1g2sp['params'].apply(lambda x: x['DEC0']).values)
data_hg1g2sp['HG1G2sp_dDEC0'] = np.degrees(data_hg1g2sp['params'].apply(lambda x: x['errDEC0']).values)
data_hg1g2sp['HG1G2sp_R'] = data_hg1g2sp['params'].apply(lambda x: x['R']).values
data_hg1g2sp['HG1G2sp_dR'] = data_hg1g2sp['params'].apply(lambda x: x['errR']).values

data_hg1g2sp['HG1G2sp_rms'] = data_hg1g2sp['params'].apply(lambda x: x['rms']).values
data_hg1g2sp['HG1G2sp_chi2red'] = data_hg1g2sp['params'].apply(lambda x: x['chi2red']).values
# data_hg1g2sp['HG1G2sp_minphase'] = np.rad2deg(data_hg1g2sp['params'].apply(lambda x: x['minphase']).values)
# data_hg1g2sp['HG1G2sp_maxphase'] = np.rad2deg(data_hg1g2sp['params'].apply(lambda x: x['maxphase']).values)
# data_hg1g2sp['HG1G2sp_n_days'] = data_hg1g2sp['ndays']
# data_hg1g2sp['HG1G2sp_n_obs'] = data_hg1g2sp['nmeasurements']

data_hg1g2sp = data_hg1g2sp.drop(columns=['nmeasurements', 'ndays', 'params'])

In [9]:
# HG1G2hyb to data
for filt in filters.keys():
    data_hg1g2hyb['HG1G2hyb_H_{}'.format(filters[filt])] = data_hg1g2hyb['params'].apply(lambda x: x['H_{}'.format(filt)]).values
    data_hg1g2hyb['HG1G2hyb_dH_{}'.format(filters[filt])] = data_hg1g2hyb['params'].apply(lambda x: x['errH_{}'.format(filt)]).values
    data_hg1g2hyb['HG1G2hyb_G1_{}'.format(filters[filt])] = data_hg1g2hyb['params'].apply(lambda x: x['G1_{}'.format(filt)]).values
    data_hg1g2hyb['HG1G2hyb_dG1_{}'.format(filters[filt])] = data_hg1g2hyb['params'].apply(lambda x: x['errG1_{}'.format(filt)]).values
    data_hg1g2hyb['HG1G2hyb_G2_{}'.format(filters[filt])] = data_hg1g2hyb['params'].apply(lambda x: x['G2_{}'.format(filt)]).values
    data_hg1g2hyb['HG1G2hyb_dG2_{}'.format(filters[filt])] = data_hg1g2hyb['params'].apply(lambda x: x['errG2_{}'.format(filt)]).values
    

data_hg1g2hyb['HG1G2hyb_RA0'] = np.degrees(data_hg1g2hyb['params'].apply(lambda x: x['alpha0']).values)
data_hg1g2hyb['HG1G2hyb_dRA0'] = np.degrees(data_hg1g2hyb['params'].apply(lambda x: x['erralpha0']).values)
data_hg1g2hyb['HG1G2hyb_DEC0'] = np.degrees(data_hg1g2hyb['params'].apply(lambda x: x['delta0']).values)
data_hg1g2hyb['HG1G2hyb_dDEC0'] = np.degrees(data_hg1g2hyb['params'].apply(lambda x: x['errdelta0']).values)
data_hg1g2hyb['HG1G2hyb_R'] = data_hg1g2hyb['params'].apply(lambda x: x['R']).values
data_hg1g2hyb['HG1G2hyb_dR'] = data_hg1g2hyb['params'].apply(lambda x: x['errR']).values

data_hg1g2hyb['HG1G2hyb_rms'] = data_hg1g2hyb['params'].apply(lambda x: x['rms']).values
data_hg1g2hyb['HG1G2hyb_chi2red'] = data_hg1g2hyb['params'].apply(lambda x: x['chi2red']).values
data_hg1g2hyb['HG1G2hyb_n_oppo'] = data_hg1g2hyb['params'].apply(lambda x: x['noppositions']).values
data_hg1g2hyb['HG1G2hyb_status'] = data_hg1g2hyb['params'].apply(lambda x: x['status']).values
data_hg1g2hyb['HG1G2hyb_fit'] = data_hg1g2hyb['params'].apply(lambda x: x['fit']).values
# data_hg1g2hyb['HG1G2hyb_minphase'] = np.rad2deg(data_hg1g2hyb['params'].apply(lambda x: x['minphase']).values)
# data_hg1g2hyb['HG1G2hyb_maxphase'] = np.rad2deg(data_hg1g2hyb['params'].apply(lambda x: x['maxphase']).values)
# data_hg1g2hyb['HG1G2hyb_n_days'] = data_hg1g2hyb['ndays']
# data_hg1g2hyb['HG1G2hyb_n_obs'] = data_hg1g2hyb['nmeasurements']

data_hg1g2hyb = data_hg1g2hyb.drop(columns=['nmeasurements', 'ndays', 'params'])


In [10]:
# HG with HG1G2
data_2 = data_hg.merge( data_hg1g2, on='ssnamenr')

# (HG with HG1G2) wth HG1G2spin
data_3 = data_2.merge( data_hg1g2sp, on='ssnamenr')

# ((HG with HG1G2) wth HG1G2spin) with hybrid
data = data_3.merge( data_hg1g2hyb, on='ssnamenr')

In [11]:
len(data), len(data_hg), len(data_hg12),  len(data_hg1g2),  len(data_hg1g2sp),  len(data_hg1g2hyb)

(104165, 104165, 104242, 104242, 104242, 104242)

In [12]:
data = data.astype({'number':'Int64'})
data

,ssnamenr,number,name,minphase,maxphase,n_days,n_obs,HG_H,HG_dH,HG_G,...,HG1G2hyb_dRA0,HG1G2hyb_DEC0,HG1G2hyb_dDEC0,HG1G2hyb_R,HG1G2hyb_dR,HG1G2hyb_rms,HG1G2hyb_chi2red,HG1G2hyb_n_oppo,HG1G2hyb_status,HG1G2hyb_fit
0,3378,3378,Susanvictoria,7.209902,27.973639,694.757685,163,13.835591,0.073765,0.186857,...,9.557656,-12.798546,17.716061,0.703476,0.037446,0.157521,15.592551,5.0,2.0,0.0
1,66214,66214,1999 CO36,2.763971,24.397613,972.951458,73,15.128266,0.062540,0.027824,...,29.987010,57.333735,17.228420,0.584858,0.294525,0.226982,6.271087,6.0,2.0,0.0
2,30141,30141,Nelvenzon,3.434811,23.955449,621.800359,53,16.165762,0.136274,0.613963,...,3.850597,57.829176,9.215828,0.100000,0.459288,0.248674,10.099229,5.0,2.0,0.0
3,161891,161891,2007 DW10,1.758005,21.985170,632.723530,61,15.447201,0.046601,0.117947,...,44.012971,-74.451227,5.611651,0.100000,0.724887,0.118081,1.002377,2.0,2.0,0.0
4,85970,85970,Fundacaoterra,0.822790,22.376936,998.833889,54,15.927574,0.101270,0.443576,...,250.401893,41.637127,359.407232,0.874673,0.914385,0.241814,4.502612,6.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104160,64758,64758,2001 XL163,1.497470,29.740566,608.827500,79,15.778768,0.072580,0.114509,...,14.982418,51.910852,12.906577,0.100000,0.394077,0.140667,2.366890,4.0,2.0,0.0
104161,76784,76784,2000 KR71,1.830102,25.589995,990.106308,82,15.468660,0.065853,0.093753,...,9.315215,-13.761004,42.692961,0.770202,0.122511,0.217536,4.852270,5.0,2.0,0.0
104162,181960,181960,1999 UM24,1.875814,18.193628,973.847419,107,14.037718,0.076177,0.000482,...,11.998336,-47.884875,44.613792,0.862264,0.068049,0.175417,3.266537,5.0,2.0,0.0
104163,366616,366616,2003 NP4,26.050772,37.302272,242.663241,153,16.717611,0.113371,0.149370,...,24.838405,-8.836291,26.896569,0.792875,0.229879,0.118694,2.036077,2.0,2.0,0.0


In [13]:
data.to_parquet(f'{data_fink}data/sso_bft_50_inf_allsolutions.parquet')

In [14]:
data.columns

Index(['ssnamenr', 'number', 'name', 'minphase', 'maxphase', 'n_days', 'n_obs',
       'HG_H', 'HG_dH', 'HG_G', 'HG_dG', 'HG_rms', 'HG_chi2red', 'HG1G2_H',
       'HG1G2_dH', 'HG1G2_G1', 'HG1G2_dG1', 'HG1G2_G2', 'HG1G2_dG2',
       'HG1G2_rms', 'HG1G2_chi2red', 'HG1G2sp_H', 'HG1G2sp_dH', 'HG1G2sp_G1',
       'HG1G2sp_dG1', 'HG1G2sp_G2', 'HG1G2sp_dG2', 'HG1G2sp_RA0',
       'HG1G2sp_dRA0', 'HG1G2sp_DEC0', 'HG1G2sp_dDEC0', 'HG1G2sp_R',
       'HG1G2sp_dR', 'HG1G2sp_rms', 'HG1G2sp_chi2red', 'HG1G2hyb_H_g',
       'HG1G2hyb_dH_g', 'HG1G2hyb_G1_g', 'HG1G2hyb_dG1_g', 'HG1G2hyb_G2_g',
       'HG1G2hyb_dG2_g', 'HG1G2hyb_H_r', 'HG1G2hyb_dH_r', 'HG1G2hyb_G1_r',
       'HG1G2hyb_dG1_r', 'HG1G2hyb_G2_r', 'HG1G2hyb_dG2_r', 'HG1G2hyb_H_o',
       'HG1G2hyb_dH_o', 'HG1G2hyb_G1_o', 'HG1G2hyb_dG1_o', 'HG1G2hyb_G2_o',
       'HG1G2hyb_dG2_o', 'HG1G2hyb_H_c', 'HG1G2hyb_dH_c', 'HG1G2hyb_G1_c',
       'HG1G2hyb_dG1_c', 'HG1G2hyb_G2_c', 'HG1G2hyb_dG2_c', 'HG1G2hyb_RA0',
       'HG1G2hyb_dRA0', 'HG1G2hyb_DEC